# Ingegneria delle caratteristiche

Finora la maggior parte degli esempi presupponevano che i dati numerici erano in un formato ordinato [n_samples, n_features]. Nel mondo reale, i dati raramente arrivano in questa forma. Con questo in mente, uno dei passaggi più importanti nell'utilizzo pratico del machine learning è l'ingegneria delle caratteristiche : ovvero, prendere tutte le informazioni che abbiamo sul nostro problema e trasformarle in numeri che possiamo utilizzare per costruire la matrice delle caratteristiche.

Andremo ora a trattare alcuni esempi comuni di attività di ingegneria delle funzionalità: funzionalità per rappresentare dati categoriali , funzionalità per rappresentare testo e funzionalità per rappresentare immagini . Inoltre, discuteremo le funzionalità derivate per aumentare la complessità del modello e l'imputazione dei dati mancanti. Spesso questo processo è noto come vettorizzazione , poiché comporta la conversione di dati arbitrari in vettori ben funzionanti.

## Caratteristiche categoriche

Un tipo comune di dati non numerici sono i dati categoriali, a esempio, immaginiamo di esplorare alcuni dati sui prezzi delle case, insieme a caratteristiche numeriche come "prezzo" e "camere", di avere anche informazioni sul "quartiere": 

In [ ]:
data = [
    {'price': 850000, 'rooms': 4, 'neighborhood': 'Queen Anne'},
    {'price': 700000, 'rooms': 3, 'neighborhood': 'Fremont'},
    {'price': 650000, 'rooms': 3, 'neighborhood': 'Wallingford'},
    {'price': 600000, 'rooms': 2, 'neighborhood': 'Fremont'}
]


In [ ]:
import pandas as pd
df =pd.DataFrame(data)
df

Potremmo essere tentato di codificare questi dati con una semplice mappatura numerica:

In [ ]:
{'Queen Anne': 1, 'Fremont': 2, 'Wallingford': 3}

Per questo ci può aiutare LabelEncoder: converte stringhe (o etichette categoriali) in
valori numerici

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x_encoded = le.fit_transform(df["neighborhood"])
x_encoded

Questo però non è generalmente un approccio utile in Scikit-Learn: i modelli del pacchetto partono dal presupposto fondamentale che le caratteristiche numeriche riflettono quantità algebriche. Pertanto una tale mappatura implicherebbe, ad esempio, che Queen Anne < Fremont < Wallingford , o anche che Wallingford - Queen Anne = Fremont , il che non ha molto senso.

In questo caso, una tecnica collaudata consiste nell'utilizzare la codifica one-hot , che crea effettivamente colonne aggiuntive che indicano la presenza o l'assenza di una categoria con un valore rispettivamente di 1 o 0. Quando i nostri dati arrivano come un elenco di dizionari, Scikit-Learn DictVectorizer li trasformerà in vettori per noi:

In [ ]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse=False, dtype=int)
vec.fit_transform(data)

Notiamo che la colonna "quartiere" è stata espansa in tre colonne separate, che rappresentano le tre etichette di quartiere, e che ciascuna riga ha un 1 nella colonna associata al suo quartiere. Con queste caratteristiche categoriche così codificate, possiamo procedere normalmente con l'adattamento di un modello Scikit-Learn.

Per vedere il significato di ciascuna colonna, possiamo controllare i nomi delle funzionalità:

In [ ]:
vec.get_feature_names_out()

C'è un chiaro svantaggio di questo approccio: se la nostra categoria ha molti valori possibili, ciò può aumentare notevolmente la dimensione del set di dati. Tuttavia, poiché i dati codificati contengono principalmente zeri, un output sparso può essere una soluzione molto efficiente:

In [ ]:
vec = DictVectorizer(sparse=True, dtype=int)
X =vec.fit_transform(data)
X

Con Pandas possiamo poi visualizzare correttamente questa matrice sparsa

In [ ]:
import pandas as pd
pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())

Molti stimatori di Scikit-Learn accettano input così sparsi quando adattano e valutano i modelli. sklearn.preprocessing.OneHotEncoder e sklearn.feature_extraction.FeatureHasher sono due strumenti aggiuntivi che Scikit-Learn inclusi per supportare questo tipo di codifica.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [ ]:
x=[["casa","no"],["non casa","si"],["casa","no"],["non casa","si"]]
y =[0,1,0,1]

X = pd.DataFrame(x)
y =pd.Series(y)
X,y

In [ ]:
# Preprocessing: encoder per colonna categorica
one_hot= OneHotEncoder(sparse_output=False)

X=one_hot.fit_transform(X)
print(X)

In [ ]:
logreg = LogisticRegression(C=1e5)
logreg.fit(X, y)

In [ ]:
y_pred =logreg.predict(X)
y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
var =classification_report(y,y_pred)

In [ ]:
confusion_matrix(y,y_pred)

## Caratteristiche del testo

Un'altra esigenza comune nell'ingegneria delle funzionalità è convertire il testo in un insieme di valori numerici rappresentativi. A esempio, la maggior parte dell’estrazione automatica dei dati dei social media si basa su una qualche forma di codifica del testo come numeri, uno dei metodi più semplici per codificare i dati è il conteggio delle parole : prendiamo ogni frammento di testo, contiamo le occorrenze di ogni parola al suo interno e inseriamo i risultati in una tabella.

Partiamo da queste tre frasi:

In [ ]:
sample = ['problem of evil',
          'evil queen',
          'horizon problem']

Per una vettorizzazione di questi dati basata sul conteggio delle parole, potremmo costruire una colonna che rappresenti la parola "problem", la parola "evil", la parola "horizon" e così via. Sebbene farlo manualmente sia possibile, la noia può essere evitata utilizzando Scikit-Learn CountVectorizer:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
X = vec.fit_transform(sample)
X

Il risultato è una matrice sparsa che registra il numero di volte in cui appare ciascuna parola; è più facile da controllare se lo convertiamo in un file DataFramecon colonne etichettate:

In [ ]:
import pandas as pd
pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())

Ci sono tuttavia alcuni problemi con questo approccio: i conteggi grezzi delle parole portano a caratteristiche che danno troppo peso alle parole che appaiono molto frequentemente, e questo può non essere ottimale in alcuni algoritmi di classificazione. Un approccio per risolvere questo problema è noto come frequenza del documento inversa alla frequenza ( TF-IDF ) che pondera il conteggio delle parole in base alla frequenza con cui appaiono nei documenti. La sintassi per calcolare queste funzionalità è simile all'esempio precedente:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X = vec.fit_transform(sample)
pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())

## Funzionalità derivate

Un altro tipo di funzionalità utile è quella derivata matematicamente da alcune funzionalità di input. Ne abbiamo visto un esempio in Iperparametri e convalida del modello quando abbiamo costruito funzionalità polinomiali dai nostri dati di input. Abbiamo visto che potremmo convertire una regressione lineare in una regressione polinomiale non cambiando il modello, ma trasformando l'input!

A esempio, questi dati chiaramente non possono essere ben descritti da una linea retta:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

x = np.array([1, 2, 3, 4, 5])
y = np.array([4, 2, 1, 3, 7])
plt.scatter(x, y)

Tuttavia, possiamo adattare una linea ai dati utilizzando la LinearRegression e ottenere il risultato ottimale:

In [ ]:
from sklearn.linear_model import LinearRegression
X = x[:, np.newaxis]
model = LinearRegression().fit(X, y)
yfit = model.predict(X)
plt.scatter(x, y)
plt.plot(x, yfit)

È chiaro che abbiamo bisogno di un modello più sofisticato per descrivere la relazione tra X
e y.

Un approccio a questo problema consiste nel trasformare i dati, aggiungendo ulteriori colonne di funzionalità per garantire una maggiore flessibilità nel modello. A esempio, possiamo aggiungere caratteristiche polinomiali ai dati in questo modo:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, include_bias=False)
X2 = poly.fit_transform(X)
print(X2)

La matrice delle caratteristiche derivate ha una colonna che rappresenta X, una seconda colonna che rappresenta X2 e una terza colonna che rappresenta X3. Il calcolo di una regressione lineare su questo input espanso fornisce un adattamento molto più vicino ai nostri dati:

In [ ]:
model = LinearRegression().fit(X2, y)
yfit = model.predict(X2)
plt.scatter(x, y)
plt.plot(x, yfit)

## Gestione Dati mancanti

Un'altra esigenza comune nell'ingegneria delle funzionalità è la gestione dei dati mancanti. Abbiamo già discusso la gestione dei dati mancanti e abbiamo visto che spesso il valore NaN  viene utilizzato per contrassegnare i valori mancanti. A esempio, potremmo avere un set di dati simile a questo:

In [ ]:
from numpy import nan
X = np.array([[ nan, 0,   3  ],
              [ 3,   7,   9  ],
              [ 3,   5,   2  ],
              [ 4,   nan, 6  ],
              [ 8,   8,   1  ]])
y = np.array([14, 16, -1,  8, -5])

Quando applichiamo un tipico modello di machine learning a tali dati, dovremo prima sostituire i dati mancanti con un valore di riempimento appropriato. Questo è noto come gestione dei valori mancanti e le strategie vanno da semplici (ad esempio, sostituire i valori mancanti con la media della colonna) a sofisticate (ad esempio, utilizzando il completamento della matrice o un modello robusto per gestire tali dati).

Gli approcci sofisticati tendono ad essere molto specifici per l'applicazione e non li approfondiremo. Per un approccio di imputazione di base, utilizzando la media, la mediana o il valore più frequente, Scikit-Learn fornisce la classe Imputer:

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='mean')
X2 = imp.fit_transform(X)
X2

Vediamo che nei dati risultanti i due valori mancanti sono stati sostituiti con la media dei valori rimanenti nella colonna. Questi dati imputati possono quindi essere inseriti direttamente, ad esempio, in uno stimatore LinearRegression:

In [ ]:
model = LinearRegression().fit(X2, y)
model.predict(X2)

## Pipeline di funzionalità

Con uno qualsiasi degli esempi precedenti, può diventare rapidamente noioso eseguire le trasformazioni manualmente, soprattutto se si desidera mettere insieme più passaggi. A esempio, potremmo volere una pipeline di elaborazione simile a questa:

- Assegnazione valori mancanti utilizzando la media;
- Trasformazione delle caratteristiche in quadratiche;
- Adattamento di una regressione lineare.
Per semplificare questo tipo di pipeline di elaborazione, Scikit-Learn fornisce un oggetto Pipeline che può essere utilizzato come segue:

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(SimpleImputer(strategy='mean'),
                      PolynomialFeatures(degree=2),
                      LinearRegression())

Questa pipeline appare e si comporta come un oggetto Scikit-Learn standard e applicherà tutti i passaggi specificati a qualsiasi dato di input.

In [ ]:
model.fit(X, y)  # X with missing values, from above
print(y)
print(model.predict(X))

Come potete vedere tutti i passaggi del modello vengono applicati automaticamente.